In [17]:
# individual case output test
import os
import numpy as np
from lake_functions import *
from lake_cases import *
import datetime as dtt
import matplotlib.pyplot as plt

# PARAMETERS
#Time
nyears=2
tyear=np.tile(np.arange(1,366,1),[1,nyears]) # DOY
tyear=tyear[0];
t=np.arange(1,365*nyears+1,1) # Time in days from the first day

# Meteorological data:
meteodata_NASA =np.genfromtxt("Meteo_data_NASA.txt", usecols=tuple(np.arange(3,9,1)),skip_header=1,dtype=float)
meteodata_NASA=np.tile(meteodata_NASA,[nyears,1]) 
Ta=meteodata_NASA[:,4]  # [°C]
RH=meteodata_NASA[:,2]  # [#]
Wsp=meteodata_NASA[:,5]  # [m/s]
R=meteodata_NASA[:,1]*10**6/(24*3600)  # [W/m^2]
P=meteodata_NASA[:,3]*1000  # [Pa]

# Atmospheric variables:
rho_air=1.2  # [kg/m^3], air density
CD=0.001  # [-], drag coefficient

#Basin:
hmax=8; # [m], lake depth

# Heat fluxes:
Twinter=4
Adiff=0.066  # [-], albedo of diffuse shortwave radiation
Adir=0.1  # [-], albedo of direct shortwave radiation (approximate value for 50°N)
C=0.5  # Cloudiness
Fdir=(1-C)/((1-C)+0.5*C) 
Fdiff=0.5*C/((1-C)+0.5*C) 
Hsw0=-R*(Fdir*(1-Adir)+Fdiff*(1-Adiff))  # [W/m^2], shortwave radiation (<0) passing through the lake surface

# Gravity
g=9.81 # [m.s^(-2)]

# Water and particles properties:
Dp=1.6*10**(-6)  # [m]
rho_p=2650 # [kg/m^3]
nu=1E-6  # [m^2/s]
Vs=g*(rho_p-1000)/1000*Dp**2/(18*nu)  # [m/s], settling velocity
f_fft=10  # [%], solids content in the FFT
C_FFT=f_fft*10**4  # [g/m^3], solids concentration in the FFT

# THERMOCLINE

iceoff=(dtt.datetime(2015,4,26)-dtt.datetime(2015,1,1)).days  # iceoff DOY (based on observations)
iceon=max(t)   # initial value for the iceon DOY, it will be calculated by the simulation
h_epi=np.zeros(np.shape(t)); h_epi[0]=hmax 

# INITIALIZATION

 # Temperature:
T_epi=np.zeros(np.shape(t)); T_epi[0]=Twinter   # [°C]
T_hypo=np.zeros(np.shape(t)); T_hypo[0]=Twinter   # [°C]

 # Solids concentration:
Css_epi=np.zeros(np.shape(t)); Css_epi[0]=100   # [mg/L]
Css_hypo=np.zeros(np.shape(t)); Css_hypo[0]=100   # [mg/L]

#initialize m_* variables:
m_add=np.zeros(np.shape(t));
m_sedi=np.zeros(np.shape(t));
m_sedi_hypo=np.zeros(np.shape(t));

In [18]:
#case 1
import numpy as np
import sys
from lake_functions import alphaT,rho_TCss, seasonal_therm
from statistics import mean

def case1(VAR_LAKE,Twinter=4,hmax=8):
    """
    Winter case: ice-covered lake, no heat fluxes = constant temperatures 
    and concentrations. 
     
    INPUTS:
        VAR_LAKE: dictionary containing
            T_epi, T_hypo: epilimnion and hypolimnion temperatures [°C]
            Css_epi, Css_hypo: epilimnion and hypolimnion solids concentrations [mg/L]
            h_epi: thermocline depth [m]
        Twinter: temperature of maximum density [°C]
        hmax: lake depth [m]
        
    OUTPUTS:
        VAR_LAKE: dictionary containing the same variables than VAR_LAKE but at the next time step
    """

    # -------- CODE --------
    # Temperature
    T_epi1 = Twinter
    T_hypo1 = Twinter
    # solid concentrations
    Css_epi1 = VAR_LAKE["Css_epi"]
    Css_hypo1 = VAR_LAKE["Css_hypo"]

    h_epi1 = hmax
    
    VAR_LAKE={"T_epi":T_epi1,"T_hypo":T_hypo1,"Css_epi":Css_epi1,"Css_hypo":Css_hypo1,"h_epi":h_epi1}

    return VAR_LAKE

In [19]:
def case2(VAR_LAKE,Hsurf,Hsw0,tyear,iceon,Vs,C_FFT,hmax=8,g=9.81,Cpw=4200,Twinter=4,A0=7.8*10**6):
    # ----Initial setup----
    h_epi2 = VAR_LAKE["h_epi"]  # Thermocline depth
    T2 = VAR_LAKE["T_epi"]  # Temperature, constant
    T_hypo = VAR_LAKE["T_hypo"] # Hypolimnion temperature for stable condition
    Css2 = VAR_LAKE["Css_epi"] # Initial solid concentration constants
    Cssh_2 = VAR_LAKE["Css_hypo"]
    
    # --Redistribution--
    rho_epi = rho_TCss(T2,Css2)  # [kg/m^3]
    rho_hypo = rho_epi
    alpha2 = alphaT(T2)
    
    kd = 0.325 + 66 * Css2 / 1000  # mg/L
    Hsw_epi2 = Hsw0 * (1 - np.exp(-kd * h_epi2))
    B = alpha2*g*(Hsurf + Hsw_epi2) / (rho_epi * Cpw)

    # --stability and concentration calculate--
    if B > 0: # unstable, fully mixed, new epi. depth is constant
        hepi_new = hmax
        Qnet2 = -(Hsurf + Hsw0)
        dT2 = (Qnet2*24*60*60/(rho_epi*h_epi*Cpw))
        Tepi_new = T2 + dT2
        Thypo_new = Tepi_new
        
        if Tepi_new <= 4: #Ice formation case
            if tyear > 200:
                iceon = tyear
            else:
                print("Warning, Ice in Summer!")
            Tepi_new = Twinter
            Thypo_new = Twinter
            Csse_new = Css2
            Cssh_new = Cssh_2
        else: # no ice
            # calculate velocities
            rho_30 = rho_TCss(T2, 30*10**5)
            N_fft2 = g*(rho_30-rho_epi)/(0.5*np.mean(rho_epi,rho_30)) 
            Vr = (1+2*0.2)*B/(N_fft2*hmax)
            # water mass
            m_w0 = rho_epi*A0*hmax 
            # new settling velocity
            Vconv=0.6*(B*h_epi2)**(1/3)
            Vs_epi=Vs-Vconv    # [m/s]
            if Vs_epi<0:
                Vs_epi=0
            # new concentration
            m_sed = -Css2*Vs_epi*A0*24*60*60
            m_red = C_FFT*Vr*A0*24*60*60
            m_initial = Css2*h_epi*A0
            m_new = m_initial+m_sed+m_red
            Csse_new = m_new/(A0*hmax)
           
            if Csse_new < 0:
                print("Warning, Solid Concentration Error!")
                Csse_new = 0
            elif Vr < 0:
                print("Warning, Resettling Velocity Error!")
            Cssh_new = Csse_new
                
    else: # stable
        T_avg = T2 + Hsurf *24*3600/(Cpw*rho_epi*h_epi)
        T_surf = 2*T_avg - T_hypo
        # density of 2 layers
        rho1 = rho_TCss(T_surf,Css2)
        rho2 = rho_TCss(T_hypo,Cssh_2)
        rho_avg = (rho1+rho2)/2
        # N^2
        N_2 = g*(rho2 - rho1)/ rho_avg / h_epi # be careful with the density
        # new depth of thermocline
        hepi_new = seasonal_therm(h_epi,3600*24,B,N_2)
        # ---Identify: stratified---
        if hepi_new > (hmax - 0.1): # stratified ignored
            hepi_new = hmax
            Tepi_new = T_avg
            Thypo_new = T_avg
        else:
            T_therm = T_surf - (hepi_new/hmax)*(T_surf - T_hypo)
            Tepi_new = 0.5 * (T_therm + T_surf)
            Thypo_new = 0.5 * (T_hypo + T_therm)
            
        Csse_new = Css2
        Cssh_new = Cssh_2

    VAR_LAKE={"T_epi":Tepi_new,"T_hypo":Thypo_new,"Css_epi":Csse_new,"Css_hypo":Cssh_new,"h_epi":hepi_new}
    return VAR_LAKE, iceon

In [20]:
# main code test
# ITERATIONS
for k in np.arange(len(t)-2): 
 
    VAR_LAKE={"T_epi":T_epi[k],"T_hypo":T_hypo[k],"Css_epi":Css_epi[k],"Css_hypo":Css_hypo[k],"h_epi":h_epi[k]}
    
    if tyear[k]<iceoff or tyear[k]>iceon:  # Case 1: ice
        VAR_LAKE=case1(VAR_LAKE)
    elif h_epi[k]==hmax: # Case 2: fall turnover.
        Hsurf=surfheat(T_epi[k],Ta[k],Wsp[k],RH[k],P[k],C)
        
        VAR_LAKE,iceon=case2(VAR_LAKE,Hsurf,Hsw0[k],tyear[k],iceon,Vs,C_FFT) 
    # else: # Case 3: stratified period. 
    #     Hsurf=surfheat(T_epi[k],Ta[k],Wsp[k],RH[k],P[k],C)
    #     rho_epi=rho_TCss(T_epi[k],Css_epi[k]);
    #     ustar=np.sqrt(rho_air*CD*Wsp[k]**2/rho_epi); 
    #     VAR_LAKE,iceon,m_add_new,m_sedi_new,m_sedi_hypo_new=case3(VAR_LAKE,Hsurf,Hsw0[k],tyear[k],iceon,Vs,ustar)
    #     m_add[k+1]=m_add_new
    #     m_sedi[k+1]=m_sedi_new
    #     m_sedi_hypo[k+1]=m_sedi_hypo_new
        
    T_epi[k+1]=VAR_LAKE["T_epi"]
    T_hypo[k+1]=VAR_LAKE["T_hypo"]
    Css_epi[k+1]=VAR_LAKE["Css_epi"]
    Css_hypo[k+1]=VAR_LAKE["Css_hypo"]
    h_epi[k+1]=VAR_LAKE["h_epi"]
    
    lake_list = []
    lake_list.append(VAR_LAKE.copy())
    print(k, lake_list)
    print(iceon)

0 [{'T_epi': 4, 'T_hypo': 4, 'Css_epi': 100.0, 'Css_hypo': 100.0, 'h_epi': 8}]
730
1 [{'T_epi': 4, 'T_hypo': 4, 'Css_epi': 100.0, 'Css_hypo': 100.0, 'h_epi': 8}]
730
2 [{'T_epi': 4, 'T_hypo': 4, 'Css_epi': 100.0, 'Css_hypo': 100.0, 'h_epi': 8}]
730
3 [{'T_epi': 4, 'T_hypo': 4, 'Css_epi': 100.0, 'Css_hypo': 100.0, 'h_epi': 8}]
730
4 [{'T_epi': 4, 'T_hypo': 4, 'Css_epi': 100.0, 'Css_hypo': 100.0, 'h_epi': 8}]
730
5 [{'T_epi': 4, 'T_hypo': 4, 'Css_epi': 100.0, 'Css_hypo': 100.0, 'h_epi': 8}]
730
6 [{'T_epi': 4, 'T_hypo': 4, 'Css_epi': 100.0, 'Css_hypo': 100.0, 'h_epi': 8}]
730
7 [{'T_epi': 4, 'T_hypo': 4, 'Css_epi': 100.0, 'Css_hypo': 100.0, 'h_epi': 8}]
730
8 [{'T_epi': 4, 'T_hypo': 4, 'Css_epi': 100.0, 'Css_hypo': 100.0, 'h_epi': 8}]
730
9 [{'T_epi': 4, 'T_hypo': 4, 'Css_epi': 100.0, 'Css_hypo': 100.0, 'h_epi': 8}]
730
10 [{'T_epi': 4, 'T_hypo': 4, 'Css_epi': 100.0, 'Css_hypo': 100.0, 'h_epi': 8}]
730
11 [{'T_epi': 4, 'T_hypo': 4, 'Css_epi': 100.0, 'Css_hypo': 100.0, 'h_epi': 8}]
730
12

C:\Users\94657\AppData\Local\Temp\ipykernel_40476\1288294466.py:62: RuntimeWarning: divide by zero encountered in divide
  T_avg = T2 + Hsurf *24*3600/(Cpw*rho_epi*h_epi)


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()